In [26]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, DataGrabber
import pandas as pd
import numpy as np
import requests

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

root = find_repo_root()

In [27]:
area = 'county:*&in=state:*' 

variables = "S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E"


# interval = list(range(2010, 2023)) # it is possible to create this variable as a time series (only for ACS 5 year estimates)

year = 2022

url = f'https://api.census.gov/data/{year}/acs/acs5/subject?get={variables}&for={area}'

response = requests.get(url)

assert response.status_code == 200 # 200 means success

data = response.json()

df = pd.DataFrame(data[1:], columns=data[0])
df.columns

Index(['S0101_C01_001E', 'S0101_C01_002E', 'S0101_C01_003E', 'S0101_C01_004E',
       'S0101_C01_005E', 'S0101_C01_006E', 'S0101_C01_007E', 'S0101_C01_008E',
       'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E',
       'S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E',
       'S0101_C01_017E', 'S0101_C01_018E', 'S0101_C01_019E', 'state',
       'county'],
      dtype='object')

In [29]:
age = df.copy()

In [30]:
column_mapping = {
    'S0101_C01_001E': 'total_pop',
    'S0101_C01_002E': 'under_5',
    'S0101_C01_003E': '5_to_9',
    'S0101_C01_004E': '10_to_14',
    'S0101_C01_005E': '15_to_19',
    'S0101_C01_006E': '20_to_24',
    'S0101_C01_007E': '25_to_29',
    'S0101_C01_008E': '30_to_34',
    'S0101_C01_009E': '35_to_39',
    'S0101_C01_010E': '40_to_44',
    'S0101_C01_011E': '45_to_49',
    'S0101_C01_012E': '50_to_54',
    'S0101_C01_013E': '55_to_59',
    'S0101_C01_014E': '60_to_64',
    'S0101_C01_015E': '65_to_69',
    'S0101_C01_016E': '70_to_74',
    'S0101_C01_017E': '75_to_79',
    'S0101_C01_018E': '80_to_84',
    'S0101_C01_019E': '85_over'
}

age.rename(columns=column_mapping, inplace=True)


In [31]:
age['GeoFIPS'] = age['state'] + age['county']
age.drop(['state', 'county'], axis=1, inplace=True)
age['GeoFIPS'] = age['GeoFIPS'].astype(np.int64)

age.head()


,total_pop,under_5,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_over,GeoFIPS
0,58761,3370,3314,4376,3924,3589,3727,3689,4032,3901,4070,3811,4280,3502,2678,2623,2125,848,902,1001
1,233420,12104,12165,16137,14320,11672,12487,13251,14844,13326,14673,15304,15824,17814,15273,14409,9783,5368,4666,1003
2,24877,1299,1616,1338,1455,1494,1690,1678,1671,1430,1477,1579,1558,1666,1574,1296,1118,464,474,1005
3,22251,1157,1438,1302,1308,1121,1689,1608,1368,1654,1628,1314,1460,1470,1298,907,813,386,330,1007
4,59077,3410,3525,4135,3705,3485,3680,3334,3541,3590,3843,3898,4135,4005,3269,3026,1901,1585,1010,1009


In [32]:
common_fips = np.intersect1d(age['GeoFIPS'].unique(), gdp['GeoFIPS'].unique())
age = age[age['GeoFIPS'].isin(common_fips)]
age = age.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left')
age = age.reindex(columns=['GeoFIPS', 'GeoName'] + list(age.columns.drop(['GeoFIPS', 'GeoName'])))
age.iloc[:, 2:] = age.iloc[:, 2:].astype(float)
age.head()

,GeoFIPS,GeoName,total_pop,under_5,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,...,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_over
0,1001,"Autauga, AL",58761.0,3370.0,3314.0,4376.0,3924.0,3589.0,3727.0,3689.0,...,3901.0,4070.0,3811.0,4280.0,3502.0,2678.0,2623.0,2125.0,848.0,902.0
1,1003,"Baldwin, AL",233420.0,12104.0,12165.0,16137.0,14320.0,11672.0,12487.0,13251.0,...,13326.0,14673.0,15304.0,15824.0,17814.0,15273.0,14409.0,9783.0,5368.0,4666.0
2,1005,"Barbour, AL",24877.0,1299.0,1616.0,1338.0,1455.0,1494.0,1690.0,1678.0,...,1430.0,1477.0,1579.0,1558.0,1666.0,1574.0,1296.0,1118.0,464.0,474.0
3,1007,"Bibb, AL",22251.0,1157.0,1438.0,1302.0,1308.0,1121.0,1689.0,1608.0,...,1654.0,1628.0,1314.0,1460.0,1470.0,1298.0,907.0,813.0,386.0,330.0
4,1009,"Blount, AL",59077.0,3410.0,3525.0,4135.0,3705.0,3485.0,3680.0,3334.0,...,3590.0,3843.0,3898.0,4135.0,4005.0,3269.0,3026.0,1901.0,1585.0,1010.0


In [33]:
age.to_csv(f"{root}/data/raw/age.csv", index=False)

In [35]:
age_percentage = age.copy()
age_percentage.iloc[:, 2:] = age_percentage.iloc[:, 2:].div(age_percentage['total_pop'], axis=0) * 100
age_percentage.drop('total_pop', axis=1, inplace=True)
age_percentage.head()


,GeoFIPS,GeoName,under_5,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_over
0,1001,"Autauga, AL",5.735096,5.639795,7.447116,6.677899,6.107793,6.342642,6.277973,6.861694,6.638757,6.926363,6.485594,7.283743,5.959735,4.557445,4.463845,3.616344,1.443134,1.535032
1,1003,"Baldwin, AL",5.185503,5.211636,6.913289,6.134864,5.000428,5.349584,5.676891,6.359352,5.709022,6.286094,6.556422,6.779196,7.631737,6.543141,6.172993,4.191158,2.299717,1.998972
2,1005,"Barbour, AL",5.221691,6.49596,5.378462,5.848776,6.005547,6.793424,6.745186,6.717048,5.748282,5.937211,6.347228,6.262813,6.696949,6.327129,5.209631,4.494111,1.865177,1.905374
3,1007,"Bibb, AL",5.199766,6.462631,5.851422,5.878387,5.037976,7.59067,7.226641,6.148038,7.433374,7.316525,5.905353,6.561503,6.606445,5.833446,4.076221,3.653768,1.734753,1.483079
4,1009,"Blount, AL",5.772128,5.966789,6.99934,6.271476,5.899081,6.229159,5.643482,5.993872,6.076815,6.50507,6.598168,6.99934,6.779288,5.533456,5.122129,3.217834,2.682939,1.709633
